# Feature selection using Bat Algorithm

In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from niapy.problems import Problem
from niapy.task import Task
import warnings
from niapy.algorithms.basic import BatAlgorithm
import pandas as np
import pandas as pd
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

In [2]:
class GaussianNBFeatureSelection(Problem):
    def __init__(self, X_train, y_train, alpha=0.99):
        super().__init__(dimension=X_train.shape[1], lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train
        self.alpha = alpha

    def _evaluate(self, x):
        selected = x > 0.5
        num_selected = selected.sum()
        if num_selected == 0:
            return 1.0
        accuracy = cross_val_score(GaussianNB(), self.X_train[:, selected], self.y_train, cv=2, n_jobs=-1).mean()
        score = 1 - accuracy
        num_features = self.X_train.shape[1]
        return self.alpha * score + (1 - self.alpha) * (num_selected / num_features)

In [3]:
class LogisticRegressionFeatureSelection(Problem):
    def __init__(self, X_train, y_train, alpha=0.99):
        super().__init__(dimension=X_train.shape[1], lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train
        self.alpha = alpha

    def _evaluate(self, x):
        selected = x > 0.5
        num_selected = selected.sum()
        if num_selected == 0:
            return 1.0
        accuracy = cross_val_score(LogisticRegression(), self.X_train[:, selected], self.y_train, cv=2, n_jobs=-1).mean()
        score = 1 - accuracy
        num_features = self.X_train.shape[1]
        return self.alpha * score + (1 - self.alpha) * (num_selected / num_features)

In [4]:
class DecisionTreeClassifierFeatureSelection(Problem):
    def __init__(self, X_train, y_train, alpha=0.99):
        super().__init__(dimension=X_train.shape[1], lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train
        self.alpha = alpha

    def _evaluate(self, x):
        selected = x > 0.5
        num_selected = selected.sum()
        if num_selected == 0:
            return 1.0
        accuracy = cross_val_score(DecisionTreeClassifier(), self.X_train[:, selected], self.y_train, cv=2, n_jobs=-1).mean()
        score = 1 - accuracy
        num_features = self.X_train.shape[1]
        return self.alpha * score + (1 - self.alpha) * (num_selected / num_features)

In [5]:
class KNeighborsClassifierFeatureSelection(Problem):
    def __init__(self, X_train, y_train, alpha=0.99):
        super().__init__(dimension=X_train.shape[1], lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train
        self.alpha = alpha

    def _evaluate(self, x):
        selected = x > 0.5
        num_selected = selected.sum()
        if num_selected == 0:
            return 1.0
        accuracy = cross_val_score(KNeighborsClassifier(), self.X_train[:, selected], self.y_train, cv=2, n_jobs=-1).mean()
        score = 1 - accuracy
        num_features = self.X_train.shape[1]
        return self.alpha * score + (1 - self.alpha) * (num_selected / num_features)

In [6]:
class SVMFeatureSelection(Problem):
    def __init__(self, X_train, y_train, alpha=0.99):
        super().__init__(dimension=X_train.shape[1], lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train
        self.alpha = alpha

    def _evaluate(self, x):
        selected = x > 0.5
        num_selected = selected.sum()
        if num_selected == 0:
            return 1.0
        accuracy = cross_val_score(SVC(), self.X_train[:, selected], self.y_train, cv=2, n_jobs=-1).mean()
        score = 1 - accuracy
        num_features = self.X_train.shape[1]
        return self.alpha * score + (1 - self.alpha) * (num_selected / num_features)

In [7]:
dataset = load_breast_cancer()
X = dataset.data
y = dataset.target
feature_names = dataset.feature_names

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1234)

problem_nb = GaussianNBFeatureSelection(X_train, y_train)
problem_lr = LogisticRegressionFeatureSelection(X_train, y_train)
problem_dt = DecisionTreeClassifierFeatureSelection(X_train, y_train)
problem_knn = KNeighborsClassifierFeatureSelection(X_train, y_train)
problem_svc = SVMFeatureSelection(X_train, y_train)
#task = Task(problem, max_iters=200)

In [8]:
batdict = {}

# Bat Algorithm (1/3: Change to population size)

In [9]:
# Set saved variable for checked parameters
bestScore = 0.0
bestCriterion = None
pop_num = 0
pop_features = None

for x in range(10,100,30):
    # Naive Bayes
    task = Task(problem_nb, max_iters=200)
        
    print("-"*50)
    print("Bat algorithm: Population size = "+ str(x))
    algorithm = BatAlgorithm(population_size=x, loudness=1.0, pulse_rate=1.0, alpha=0.97, gamma=0.1, min_frequency=0.0, max_frequency=2.0)
    best_features, best_fitness = algorithm.run(task=task)
    selected_features = best_features > 0.5
    print('Number of selected features:', selected_features.sum())
    print('Selected features:', ', '.join(feature_names[selected_features].tolist()))
    features1 = feature_names[selected_features].tolist()
        
    model_selected = GaussianNB()
    model_all = GaussianNB()
    model_name = "GaussianNB"
        
    model_selected.fit(X_train[:, selected_features], y_train)
    subset_acc = model_selected.score(X_test[:, selected_features], y_test)
    print(model_name + ' - Subset accuracy:', subset_acc)        
    if subset_acc > bestScore:
        bestScore = subset_acc
        bestCriterion = x
        pop_num = selected_features.sum()
        pop_features = feature_names[selected_features].tolist()
    model_all.fit(X_train, y_train)
    print(model_name + ' - All Features Accuracy:', model_all.score(X_test, y_test))
        
    # Decision Tree
    task = Task(problem_dt, max_iters=200)
        
    print("-"*50)
    print("Bat algorithm: Population size = "+ str(x))
    algorithm = BatAlgorithm(population_size=x, loudness=1.0, pulse_rate=1.0, alpha=0.97, gamma=0.1, min_frequency=0.0, max_frequency=2.0)
    best_features, best_fitness = algorithm.run(task=task)
    selected_features = best_features > 0.5
    print('Number of selected features:', selected_features.sum())
    print('Selected features:', ', '.join(feature_names[selected_features].tolist()))
    features1 = feature_names[selected_features].tolist()
        
    model_selected = DecisionTreeClassifier()
    model_all = DecisionTreeClassifier()
    model_name = "DecisionTreeClassifier"
        
    model_selected.fit(X_train[:, selected_features], y_train)
    subset_acc = model_selected.score(X_test[:, selected_features], y_test)
    print(model_name + ' - Subset accuracy:', subset_acc)        
    if subset_acc > bestScore:
        bestScore = subset_acc
        bestCriterion = x
        pop_num = selected_features.sum()
        pop_features = feature_names[selected_features].tolist()
    model_all.fit(X_train, y_train)
    print(model_name + ' - All Features Accuracy:', model_all.score(X_test, y_test))
        
    # KNN
    task = Task(problem_knn, max_iters=200)
        
    print("-"*50)
    print("Bat algorithm: Population size = "+ str(x))
    algorithm = BatAlgorithm(population_size=x, loudness=1.0, pulse_rate=1.0, alpha=0.97, gamma=0.1, min_frequency=0.0, max_frequency=2.0)
    best_features, best_fitness = algorithm.run(task=task)
    selected_features = best_features > 0.5
    print('Number of selected features:', selected_features.sum())
    print('Selected features:', ', '.join(feature_names[selected_features].tolist()))
    features1 = feature_names[selected_features].tolist()
        
    model_selected = KNeighborsClassifier()
    model_all = KNeighborsClassifier()
    model_name = "KNeighborsClassifier"
        
    model_selected.fit(X_train[:, selected_features], y_train)
    subset_acc = model_selected.score(X_test[:, selected_features], y_test)
    print(model_name + ' - Subset accuracy:', subset_acc)        
    if subset_acc > bestScore:
        bestScore = subset_acc
        bestCriterion = x
        pop_num = selected_features.sum()
        pop_features = feature_names[selected_features].tolist()
    model_all.fit(X_train, y_train)
    print(model_name + ' - All Features Accuracy:', model_all.score(X_test, y_test))
        
    # SVC
    task = Task(problem_svc, max_iters=200)
    print("-"*50)
    print("Bat algorithm: Population size = "+ str(x))
    algorithm = BatAlgorithm(population_size=x, loudness=1.0, pulse_rate=1.0, alpha=0.97, gamma=0.1, min_frequency=0.0, max_frequency=2.0)
    best_features, best_fitness = algorithm.run(task=task)
    selected_features = best_features > 0.5
    print('Number of selected features:', selected_features.sum())
    print('Selected features:', ', '.join(feature_names[selected_features].tolist()))
    features1 = feature_names[selected_features].tolist()
    model_selected = SVC()
    model_all = SVC()
    model_name = "SVC"
    model_selected.fit(X_train[:, selected_features], y_train)
    subset_acc = model_selected.score(X_test[:, selected_features], y_test)
    print(model_name + ' - Subset accuracy:', subset_acc)        
    if subset_acc > bestScore:
        bestScore = subset_acc
        bestCriterion = x
        pop_num = selected_features.sum()
        pop_features = feature_names[selected_features].tolist()
    model_all.fit(X_train, y_train)
    print(model_name + ' - All Features Accuracy:', model_all.score(X_test, y_test))

batdict["Bat-Population"] = "pop: " + str(bestCriterion) + ", accuracy: " + str(round(bestScore, 5))

--------------------------------------------------
Bat algorithm: Population size = 10
Number of selected features: 9
Selected features: mean texture, radius error, texture error, fractal dimension error, worst texture, worst perimeter, worst area, worst compactness, worst concave points
GaussianNB - Subset accuracy: 0.9649122807017544
GaussianNB - All Features Accuracy: 0.9649122807017544
--------------------------------------------------
Bat algorithm: Population size = 10
Number of selected features: 15
Selected features: mean radius, mean perimeter, mean area, mean fractal dimension, compactness error, concavity error, concave points error, fractal dimension error, worst radius, worst texture, worst smoothness, worst concavity, worst concave points, worst symmetry, worst fractal dimension
DecisionTreeClassifier - Subset accuracy: 0.9473684210526315
DecisionTreeClassifier - All Features Accuracy: 0.9649122807017544
--------------------------------------------------
Bat algorithm: Po

In [10]:
batdict

{'Bat-Population': 'pop: 10, accuracy: 0.96491'}

# Bat Algorithm (2/3: Change to loudness)

In [11]:
# Set saved variable for checked parameters
bestScore = 0.0
bestCriterion = None
loud_num = 0
loud_features = None


for x in range(1,10,3):
    # GaussianNB
    task = Task(problem_nb, max_iters=200)
        
    print("-"*50)
    print("Bat algorithm: Loudness = "+ str(x))
    algorithm = BatAlgorithm(population_size=40, loudness=x, pulse_rate=1.0, alpha=0.97, gamma=0.1, min_frequency=0.0, max_frequency=2.0)
    best_features, best_fitness = algorithm.run(task=task)
    selected_features = best_features > 0.5
    print('Number of selected features:', selected_features.sum())
    print('Selected features:', ', '.join(feature_names[selected_features].tolist()))
    features1 = feature_names[selected_features].tolist()
        
    model_selected = GaussianNB()
    model_all = GaussianNB()
    model_name = "GaussianNB"

    model_selected.fit(X_train[:, selected_features], y_train)
        
    subset_acc = model_selected.score(X_test[:, selected_features], y_test)
    print(model_name + ' - Subset accuracy:', subset_acc)
    if subset_acc > bestScore:
        bestScore = subset_acc
        bestCriterion = x
        loud_num = selected_features.sum()
        loud_features = feature_names[selected_features].tolist()
    model_all.fit(X_train, y_train)
    print(model_name + ' - All Features Accuracy:', model_all.score(X_test, y_test))
    
    # DecisionTreeClassifier
    task = Task(problem_dt, max_iters=200)
        
    print("-"*50)
    print("Bat algorithm: Loudness = "+ str(x))
    algorithm = BatAlgorithm(population_size=40, loudness=x, pulse_rate=1.0, alpha=0.97, gamma=0.1, min_frequency=0.0, max_frequency=2.0)
    best_features, best_fitness = algorithm.run(task=task)
    selected_features = best_features > 0.5
    print('Number of selected features:', selected_features.sum())
    print('Selected features:', ', '.join(feature_names[selected_features].tolist()))
    features1 = feature_names[selected_features].tolist()
        
    model_selected = DecisionTreeClassifier()
    model_all = DecisionTreeClassifier()
    model_name = "DecisionTreeClassifier"

    model_selected.fit(X_train[:, selected_features], y_train)
        
    subset_acc = model_selected.score(X_test[:, selected_features], y_test)
    print(model_name + ' - Subset accuracy:', subset_acc)
    if subset_acc > bestScore:
        bestScore = subset_acc
        bestCriterion = x
        loud_num = selected_features.sum()
        loud_features = feature_names[selected_features].tolist()
    model_all.fit(X_train, y_train)
    print(model_name + ' - All Features Accuracy:', model_all.score(X_test, y_test))
    
    # KNeighborsClassifier
    task = Task(problem_knn, max_iters=200)
        
    print("-"*50)
    print("Bat algorithm: Loudness = "+ str(x))
    algorithm = BatAlgorithm(population_size=40, loudness=x, pulse_rate=1.0, alpha=0.97, gamma=0.1, min_frequency=0.0, max_frequency=2.0)
    best_features, best_fitness = algorithm.run(task=task)
    selected_features = best_features > 0.5
    print('Number of selected features:', selected_features.sum())
    print('Selected features:', ', '.join(feature_names[selected_features].tolist()))
    features1 = feature_names[selected_features].tolist()
        
    model_selected = KNeighborsClassifier()
    model_all = KNeighborsClassifier()
    model_name = "KNeighborsClassifier"

    model_selected.fit(X_train[:, selected_features], y_train)
        
    subset_acc = model_selected.score(X_test[:, selected_features], y_test)
    print(model_name + ' - Subset accuracy:', subset_acc)
    if subset_acc > bestScore:
        bestScore = subset_acc
        bestCriterion = x
        loud_num = selected_features.sum()
        loud_features = feature_names[selected_features].tolist()
    model_all.fit(X_train, y_train)
    print(model_name + ' - All Features Accuracy:', model_all.score(X_test, y_test))
    
    # SVC
    task = Task(problem_svc, max_iters=200)
        
    print("-"*50)
    print("Bat algorithm: Loudness = "+ str(x))
    algorithm = BatAlgorithm(population_size=40, loudness=x, pulse_rate=1.0, alpha=0.97, gamma=0.1, min_frequency=0.0, max_frequency=2.0)
    best_features, best_fitness = algorithm.run(task=task)
    selected_features = best_features > 0.5
    print('Number of selected features:', selected_features.sum())
    print('Selected features:', ', '.join(feature_names[selected_features].tolist()))
    features1 = feature_names[selected_features].tolist()
        
    model_selected = SVC()
    model_all = SVC()
    model_name = "SVC"

    model_selected.fit(X_train[:, selected_features], y_train)
        
    subset_acc = model_selected.score(X_test[:, selected_features], y_test)
    print(model_name + ' - Subset accuracy:', subset_acc)
    if subset_acc > bestScore:
        bestScore = subset_acc
        bestCriterion = x
        loud_num = selected_features.sum()
        loud_features = feature_names[selected_features].tolist()
    model_all.fit(X_train, y_train)
    print(model_name + ' - All Features Accuracy:', model_all.score(X_test, y_test))
batdict["Bat-Loudness"] = "loud: " + str(bestCriterion) + ", accuracy: " + str(round(bestScore, 5))

--------------------------------------------------
Bat algorithm: Loudness = 1
Number of selected features: 8
Selected features: mean texture, mean smoothness, mean concave points, worst radius, worst texture, worst area, worst concave points, worst symmetry
GaussianNB - Subset accuracy: 0.956140350877193
GaussianNB - All Features Accuracy: 0.9649122807017544
--------------------------------------------------
Bat algorithm: Loudness = 1
Number of selected features: 14
Selected features: mean radius, mean texture, mean area, mean smoothness, mean concave points, radius error, texture error, area error, smoothness error, compactness error, symmetry error, worst area, worst concavity, worst concave points
DecisionTreeClassifier - Subset accuracy: 0.9649122807017544
DecisionTreeClassifier - All Features Accuracy: 0.9649122807017544
--------------------------------------------------
Bat algorithm: Loudness = 1
Number of selected features: 5
Selected features: mean radius, mean texture, peri

# Bat Algorithm (3/3: Change to pulse rate)

In [12]:
# Set saved variable for checked parameters
bestScore = 0.0
bestCriterion = None
pulse_num = 0
pulse_features = None


for x in range(1,10,3):
    # GaussianNB
    task = Task(problem_nb, max_iters=200)
    
    print("-"*50)
    print("Bat algorithm: Pulse rate = "+ str(x))
    algorithm = BatAlgorithm(population_size=40, loudness=1.0, pulse_rate=x, alpha=0.97, gamma=0.1, min_frequency=0.0, max_frequency=2.0)
    best_features, best_fitness = algorithm.run(task=task)
    selected_features = best_features > 0.5
    print('Number of selected features:', selected_features.sum())
    print('Selected features:', ', '.join(feature_names[selected_features].tolist()))
    features1 = feature_names[selected_features].tolist()
    
    model_selected = GaussianNB()
    model_all = GaussianNB()
    model_name = "GaussianNB"

    model_selected.fit(X_train[:, selected_features], y_train)
    subset_acc = model_selected.score(X_test[:, selected_features], y_test)
    print(model_name + ' - Subset accuracy:', subset_acc)
    if subset_acc > bestScore:
        bestScore = subset_acc
        bestCriterion = x
        pulse_num = selected_features.sum()
        pulse_features = feature_names[selected_features].tolist()
    model_all.fit(X_train, y_train)
    print(model_name + ' - All Features Accuracy:', model_all.score(X_test, y_test))
    
    # DecisionTreeClassifier
    task = Task(problem_dt, max_iters=200)
    
    print("-"*50)
    print("Bat algorithm: Pulse rate = "+ str(x))
    algorithm = BatAlgorithm(population_size=40, loudness=1.0, pulse_rate=x, alpha=0.97, gamma=0.1, min_frequency=0.0, max_frequency=2.0)
    best_features, best_fitness = algorithm.run(task=task)
    selected_features = best_features > 0.5
    print('Number of selected features:', selected_features.sum())
    print('Selected features:', ', '.join(feature_names[selected_features].tolist()))
    features1 = feature_names[selected_features].tolist()
    
    model_selected = DecisionTreeClassifier()
    model_all = DecisionTreeClassifier()
    model_name = "DecisionTreeClassifier"

    model_selected.fit(X_train[:, selected_features], y_train)
    subset_acc = model_selected.score(X_test[:, selected_features], y_test)
    print(model_name + ' - Subset accuracy:', subset_acc)
    if subset_acc > bestScore:
        bestScore = subset_acc
        bestCriterion = x
        pulse_num = selected_features.sum()
        pulse_features = feature_names[selected_features].tolist()
    model_all.fit(X_train, y_train)
    print(model_name + ' - All Features Accuracy:', model_all.score(X_test, y_test))
    
    # KNeighborsClassifier
    task = Task(problem_knn, max_iters=200)
    
    print("-"*50)
    print("Bat algorithm: Pulse rate = "+ str(x))
    algorithm = BatAlgorithm(population_size=40, loudness=1.0, pulse_rate=x, alpha=0.97, gamma=0.1, min_frequency=0.0, max_frequency=2.0)
    best_features, best_fitness = algorithm.run(task=task)
    selected_features = best_features > 0.5
    print('Number of selected features:', selected_features.sum())
    print('Selected features:', ', '.join(feature_names[selected_features].tolist()))
    features1 = feature_names[selected_features].tolist()
    
    model_selected = KNeighborsClassifier()
    model_all = KNeighborsClassifier()
    model_name = "KNeighborsClassifier"

    model_selected.fit(X_train[:, selected_features], y_train)
    subset_acc = model_selected.score(X_test[:, selected_features], y_test)
    print(model_name + ' - Subset accuracy:', subset_acc)
    if subset_acc > bestScore:
        bestScore = subset_acc
        bestCriterion = x
        pulse_num = selected_features.sum()
        pulse_features = feature_names[selected_features].tolist()
    model_all.fit(X_train, y_train)
    print(model_name + ' - All Features Accuracy:', model_all.score(X_test, y_test))
    
    # SVC
    task = Task(problem_svc, max_iters=200)
    
    print("-"*50)
    print("Bat algorithm: Pulse rate = "+ str(x))
    algorithm = BatAlgorithm(population_size=40, loudness=1.0, pulse_rate=x, alpha=0.97, gamma=0.1, min_frequency=0.0, max_frequency=2.0)
    best_features, best_fitness = algorithm.run(task=task)
    selected_features = best_features > 0.5
    print('Number of selected features:', selected_features.sum())
    print('Selected features:', ', '.join(feature_names[selected_features].tolist()))
    features1 = feature_names[selected_features].tolist()
    
    model_selected = SVC()
    model_all = SVC()
    model_name = "SVC"

    model_selected.fit(X_train[:, selected_features], y_train)
    subset_acc = model_selected.score(X_test[:, selected_features], y_test)
    print(model_name + ' - Subset accuracy:', subset_acc)
    if subset_acc > bestScore:
        bestScore = subset_acc
        bestCriterion = x
        pulse_num = selected_features.sum()
        pulse_features = feature_names[selected_features].tolist()
    model_all.fit(X_train, y_train)
    print(model_name + ' - All Features Accuracy:', model_all.score(X_test, y_test))
batdict["Bat-Pulse"] = "pulse_rate: " + str(bestCriterion) + ", accuracy: " + str(round(bestScore, 5))

--------------------------------------------------
Bat algorithm: Pulse rate = 1
Number of selected features: 9
Selected features: mean texture, mean concave points, fractal dimension error, worst radius, worst texture, worst perimeter, worst smoothness, worst symmetry, worst fractal dimension
GaussianNB - Subset accuracy: 0.9736842105263158
GaussianNB - All Features Accuracy: 0.9649122807017544
--------------------------------------------------
Bat algorithm: Pulse rate = 1
Number of selected features: 15
Selected features: mean radius, mean texture, mean perimeter, mean concavity, mean concave points, mean symmetry, texture error, perimeter error, area error, compactness error, concave points error, symmetry error, worst texture, worst area, worst concave points
DecisionTreeClassifier - Subset accuracy: 0.9912280701754386
DecisionTreeClassifier - All Features Accuracy: 0.956140350877193
--------------------------------------------------
Bat algorithm: Pulse rate = 1
Number of selecte

# Data Preprocess

In [13]:
df = pd.DataFrame(dataset.data, columns=dataset.feature_names)
df['target'] = dataset.target
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [14]:
batdict

{'Bat-Population': 'pop: 10, accuracy: 0.96491',
 'Bat-Loudness': 'loud: 4, accuracy: 0.97368',
 'Bat-Pulse': 'pulse_rate: 1, accuracy: 0.99123'}

In [15]:
print("pop_num: {}, pop_features: {}\nloud_num: {}, loud_features: {}\npulse_num: {}, pulse_features: {}\n".format(pop_num, pop_features, loud_num, loud_features, pulse_num, pulse_features))

pop_num: 9, pop_features: ['mean texture', 'radius error', 'texture error', 'fractal dimension error', 'worst texture', 'worst perimeter', 'worst area', 'worst compactness', 'worst concave points']
loud_num: 13, loud_features: ['mean texture', 'mean perimeter', 'mean smoothness', 'mean concave points', 'texture error', 'perimeter error', 'area error', 'compactness error', 'concave points error', 'worst area', 'worst concavity', 'worst concave points', 'worst fractal dimension']
pulse_num: 15, pulse_features: ['mean radius', 'mean texture', 'mean perimeter', 'mean concavity', 'mean concave points', 'mean symmetry', 'texture error', 'perimeter error', 'area error', 'compactness error', 'concave points error', 'symmetry error', 'worst texture', 'worst area', 'worst concave points']



In [16]:
newdf = df.drop(columns=[col for col in df if col not in pop_features])
newdf.head()

,mean texture,radius error,texture error,fractal dimension error,worst texture,worst perimeter,worst area,worst compactness,worst concave points
0,10.38,1.0950,0.9053,0.006193,17.33,184.60,2019.0,0.6656,0.2654
1,17.77,0.5435,0.7339,0.003532,23.41,158.80,1956.0,0.1866,0.1860
2,21.25,0.7456,0.7869,0.004571,25.53,152.50,1709.0,0.4245,0.2430
3,20.38,0.4956,1.1560,0.009208,26.50,98.87,567.7,0.8663,0.2575
4,14.34,0.7572,0.7813,0.005115,16.67,152.20,1575.0,0.2050,0.1625


In [17]:
y =  dataset.target
X = newdf
print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 21)

#Feature Scaling, this is important as itt will allow the algorithm to quickly learn a better solution.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

(569, 9) (569,)


# Naive Bayes Model

In [18]:
def gaussianNB():
    
    from sklearn.naive_bayes import GaussianNB
    gnb = GaussianNB()
    gnb.fit(X_train,y_train)
    y_pred_test = gnb.predict(X_test)
    from sklearn.metrics import accuracy_score
    acc = accuracy_score(y_test,y_pred_test)
    print("Naive Bayes : ", acc)
    b_class = ["benign", "malign"]
    print(classification_report(y_test, y_pred_test, target_names=b_class))
    
    return y_pred_test

# LR Model

In [19]:
def logisticRegression():
    from sklearn.linear_model import LogisticRegression
    logreg = LogisticRegression(solver = 'liblinear',multi_class='auto')
    logreg.fit(X_train,y_train)
    y_pred = logreg.predict(X_test)
    from sklearn.metrics import accuracy_score
    acc1 = accuracy_score(y_test,y_pred)
    print("Logistic Regression: ", acc1)
    b_class = ["benign", "malign"]
    print(classification_report(y_test, y_pred, target_names=b_class))
    
    return y_pred

# Decision Tree Model

In [20]:
def decisionTrees():
    from sklearn.tree import DecisionTreeClassifier
    dt = DecisionTreeClassifier()
    dt.fit(X_train,y_train)
    y_pred2 = dt.predict(X_test)
    acc2 = accuracy_score(y_test,y_pred2)
    print(acc2)
    print("Decision Trees:", acc2)
    b_class = ["benign", "malign"]
    print(classification_report(y_test, y_pred2, target_names=b_class))

    return y_pred2

# KNN Model

In [21]:
def knn():
    
    from sklearn.neighbors import KNeighborsClassifier
    clf = KNeighborsClassifier(n_neighbors=8,algorithm='ball_tree')
    clf.fit(X_train,y_train)
    y_pred3 = clf.predict(X_test)
    acc3 =   accuracy_score(y_test,y_pred3)
    print("K Nearest Neighbors: ", acc3)
    b_class = ["benign", "malign"]
    print(classification_report(y_test, y_pred3, target_names=b_class))  
    
    return y_pred3

# Support Vector Model

In [22]:
def svc():
    
    from sklearn.svm import SVC
    svc1 = SVC(C=50,kernel='rbf',gamma=1)     
    svc1.fit(X_train,y_train)
    y_pred4 = svc1.predict(X_test)
    from sklearn.metrics import accuracy_score
    acc4=    accuracy_score(y_test,y_pred4)
    print("Support Vector Classifier Accuracy", acc4)
    b_class = ["benign", "malign"]
    print(classification_report(y_test, y_pred4, target_names=b_class))  
    
    return y_pred4

# Model Results

In [23]:
models = {
        "gaussianNB": gaussianNB(),
        "decisionTrees": decisionTrees() ,
        "logisticRegression": logisticRegression(),
        "svc": svc(),
        "knn": knn()
    }

results = {}
print("\nNow testing algorithms")
for algo in models:

    print("trained model: ", algo )

print("\n")

Naive Bayes :  0.9473684210526315
              precision    recall  f1-score   support

      benign       0.95      0.90      0.92        39
      malign       0.95      0.97      0.96        75

    accuracy                           0.95       114
   macro avg       0.95      0.94      0.94       114
weighted avg       0.95      0.95      0.95       114

0.9035087719298246
Decision Trees: 0.9035087719298246
              precision    recall  f1-score   support

      benign       0.91      0.79      0.85        39
      malign       0.90      0.96      0.93        75

    accuracy                           0.90       114
   macro avg       0.91      0.88      0.89       114
weighted avg       0.90      0.90      0.90       114

Logistic Regression:  0.9736842105263158
              precision    recall  f1-score   support

      benign       0.97      0.95      0.96        39
      malign       0.97      0.99      0.98        75

    accuracy                           0.97       114

In [24]:
batdict

{'Bat-Population': 'pop: 10, accuracy: 0.96491',
 'Bat-Loudness': 'loud: 4, accuracy: 0.97368',
 'Bat-Pulse': 'pulse_rate: 1, accuracy: 0.99123'}